In [1]:
from google.colab import drive
import os
drive.mount('/content/drive')
root = '/content/drive/My Drive/_project'
os.chdir(root)

Mounted at /content/drive


In [8]:
# Load libraries

import os
import numpy as np
import pretty_midi
from reverse_pianoroll import *
from collections import Counter

In [6]:
# User parameters
os.chdir('/content/drive/My Drive/_project/AutoTranscriber-recordings')
path = os.getcwd()
sampling_frequency = 31 # sampling rate for MIDI files
partition_size = 20 # number of seconds each partition of the song should be

In [9]:
wav_path = '/content/drive/My Drive/_project/AutoTranscriber-recordings/adjusted/spec'
wav_list = []
for file in os.listdir(wav_path):
    if file.endswith(".npy"):
        wav_list.append(file)
wav_list.sort()

name_list = []
for wav_file in wav_list:
  name, u = wav_file.split('_a')
  name_list.append(name)
# Dictionary containing the frequency of each wav file (from spectogram)
count_dict = dict(Counter(name_list))


['Air_1_adjusted_1.npy', 'Aria_1_adjusted_1.npy', 'CanonInD_17_adjusted_1.npy', 'CanonInD_17_adjusted_2.npy', 'CanonInD_1_adjusted_1.npy', 'CanonInD_37_adjusted_1.npy', 'CanonInD_37_adjusted_2.npy', 'Dora_1_adjusted_1.npy', 'Dora_1_adjusted_2.npy', 'Elmo_1_adjusted_1.npy', 'Elmo_1_adjusted_2.npy', 'Elvis_1_adjusted_1.npy', 'Elvis_1_adjusted_2.npy', 'Faded_1_adjusted_1.npy', 'Fantasia(Handel)_1_adjusted_1.npy', 'Fantasia(Handel)_1_adjusted_2.npy', 'Fantasia(Handel)_30_adjusted_1.npy', 'Fantasia(Handel)_30_adjusted_2.npy', 'FantasiaTelemann_1_adjusted_1.npy', 'FantasiaTelemann_1_adjusted_2.npy', 'FolkDance_1_adjusted_1.npy', 'FolkDance_1_adjusted_2.npy', 'Gymnopedie_17_adjusted_1.npy', 'Gymnopedie_17_adjusted_2.npy', 'Gymnopedie_17_adjusted_3.npy', 'Gymnopedie_1_adjusted_1.npy', 'Gymnopedie_1_adjusted_2.npy', 'Gymnopedie_37_adjusted_1.npy', 'Gymnopedie_37_adjusted_2.npy', 'Gymnopedie_37_adjusted_3.npy', 'Gymnopedie_37_adjusted_4.npy', 'Gymnopedie_37_adjusted_5.npy', 'Gymnopedie_37_adjust

In [17]:
mid_list = []
for file in os.listdir(path):
    if file.endswith(".mid"):
        mid_list.append(file)
mid_list.sort()

['Air_1.mid', 'Aria_1.mid', 'CanonInD_1.mid', 'CanonInD_17.mid', 'CanonInD_37.mid', 'Dora_1.mid', 'Elmo_1.mid', 'Elvis_1.mid', 'Faded_1.mid', 'Fantasia(Handel)_1.mid', 'FantasiaTelemann_1.mid', 'FolkDance_1.mid', 'Gymnopedie_1.mid', 'Gymnopedie_17.mid', 'Gymnopedie_37.mid', 'Lovely_1.mid', 'Lovely_44.mid', 'MinuetFHaydn_1.mid', 'MinuetFMozart_1.mid', 'MinuetteInG_1.mid', 'MoonlightSonata_1.mid', 'MoonlightSonata_20.mid', 'MoonlightSonata_35.mid', 'MoonlightSonata_44.mid', 'MorningMood_1.mid', 'NocturneDflat_1.mid', 'NocturneDflat_16.mid', 'OCanada_1.mid', 'Prelude(Bach)_1.mid', 'Prelude(Bach)_13.mid', 'PreludeinGminor_1.mid', 'Quadrille_1.mid', 'Spongebob_1.mid', 'TheLark_1.mid', 'TheLark_13.mid']
35


In [31]:
new_path = os.path.join(path+'/midi_crop/')

for file in mid_list:
  mid_name, ext = os.path.splitext(file)
  #print(mid_name)
  # Load MIDI file
  midi = pretty_midi.PrettyMIDI(os.path.join(path,file))
  piano_notes = midi.get_piano_roll(fs=sampling_frequency)
        
  # Get each partition of the MIDI and save it
  num_partitions = int(piano_notes.shape[1]/sampling_frequency/partition_size)

  # Only crop the file for the number of partitions we need
  if num_partitions > count_dict[mid_name]:
    #print(num_partitions)
    num_partitions = count_dict[mid_name]
    #print(num_partitions)

  for i in range(num_partitions):   
    audio_start = int(i*partition_size*sampling_frequency)
    audio_end = int((i+1)*partition_size*sampling_frequency)
    piano_clip = piano_notes[:, audio_start:audio_end].astype(np.int)
    piano_clip[piano_clip > 127] = 127
    
    # convert piano roll clip to midi
    midi_clip = piano_roll_to_pretty_midi(piano_clip, fs=sampling_frequency)
    midi_clip.write(os.path.join(new_path,file[:-4]+"_"+str(i+1)+".mid"))

/usr/local/lib/python3.6/dist-packages/pretty_midi/pretty_midi.py:101: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  RuntimeWarning)
